# 머신러닝 기반 영화 리뷰 감성 분석
1. 데이터 준비 (전처리된 데이터)
2. 특징 벡터 추출 (TF-IDF)
3. 머신러닝 모델별 학습 및 평가
4. 영화 리뷰 긍부정 판단

## 1.데이터 준비 (전처리된 데이터)

In [1]:
data_filename = 'Korean_movie_reviews_2016.csv'

# 데이터 로딩
import pandas as pd
review_df = pd.read_csv("./data/" + data_filename)
review_df.head()

review  label
0                                  부산 행 때문 너무 기대하고 봤      0
1                           한국 좀비 영화 어색하지 않게 만들어졌 놀랍      1
2                        조금 전 보고 왔 지루하다 언제 끝나 이 생각 드      0
3  평 밥 끼 먹자 돈 니 내고 미친 놈 정신사 좀 알 싶어 그래 밥 먹다 먹던 숟가락...      1
4  점수 대가 과 이 엑소 팬 어중간 점수 줄리 없겠 클레멘타인 이후 최고 평점 조작 ...      0

In [2]:
# 입력 데이터와 정답 데이터 추출
review_list = list(review_df.review)
label_list = list(review_df.label)
len(review_list), len(label_list)

(165384, 165384)

In [3]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
review_train, review_test, label_train, label_test = train_test_split(review_list, label_list, test_size=0.1)
len(review_train), len(review_test), len(label_train), len(label_test)

(148845, 16539, 148845, 16539)

## 2. 특징 벡터 추출 : TF-IDF

In [4]:
# 한국어 감성 분석용 tokenizer 정의
from konlpy.tag import Okt
def my_tokenizer(text):
    t = Okt()
    my_tags = ['Noun', 'Verb', 'Adjective']
    return [word for word, tag in t.pos(text) if tag in my_tags]

In [6]:
# 최대 단어 수 1000개
# 학습 데이터로 Vectorizer 생성 및 학습 데이터 특징 벡터 추출
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(tokenizer = my_tokenizer, max_features=1000)
tv.fit(review_train)
x_train = tv.transform(review_train)
x_train = tv.fit_transform(review_train)

C:\Users\User\anaconda3\envs\deepnlpenv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [7]:
# 테스트 데이터 특징 벡터 추출
x_test = tv.transform(review_test)
x_test.shape

(16539, 1000)

In [8]:
# 정답 데이터 변환 (np.array)
import numpy as np
y_train = np.array(label_train)
y_test = np.array(label_test)
y_train.shape, y_test.shape

((148845,), (16539,))

In [9]:
y_train[:10]

array([1, 1, 0, 1, 1, 1, 1, 0, 0, 1])

## 3. 머신러닝 모델별 학습 및 평가
* 의사결정 트리
* 랜덤포레스트
* 나이브 베이즈
* 로지스틱 회귀 분석
* SVM
* Perceptron

In [10]:
# 머신러닝 모델별 학습 성능 평가 결과 저장 준비
import pandas as pd
score_df = pd.DataFrame(columns=['train', 'test'])
score_df

Empty DataFrame
Columns: [train, test]
Index: []

## 3.1 의사결정 트리 (Decision Tree)

In [12]:
# 학습
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=100)
dtc.fit(x_train, y_train)

DecisionTreeClassifier(random_state=100)

In [13]:
# 성능 평가
train_score = dtc.score(x_train, y_train)
test_score = dtc.score(x_test, y_test)
print(train_score, test_score)

0.9810742718935805 0.8053691275167785


In [14]:
# 평가 결과 score_df에 추가
score_df.loc['DecisionTree'] = [train_score, test_score]
score_df

train      test
DecisionTree  0.981074  0.805369

## 3.2 랜덤 포레스트 (Random Forrest)

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [16]:
# 성능 평가
train_score = rf.score(x_train, y_train)
test_score = rf.score(x_test, y_test)
print(train_score, test_score)

0.9810742718935805 0.85150250922063


In [17]:
# 평가 결과 score_df에 추가
score_df.loc['RandomForest'] = [train_score, test_score]
score_df

train      test
DecisionTree  0.981074  0.805369
RandomForest  0.981074  0.851503

## 3.3 나이브 베이즈 (Naive Baysian)

In [18]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train, y_train)

MultinomialNB()

In [19]:
# 성능 평가
train_score = mnb.score(x_train, y_train)
test_score = mnb.score(x_test, y_test)
print(train_score, test_score)

0.8525647485639424 0.8530745510611283


In [20]:
# 평가 결과 score_df에 추가
score_df.loc['NaiveBayes'] = [train_score, test_score]
score_df

train      test
DecisionTree  0.981074  0.805369
RandomForest  0.981074  0.851503
NaiveBayes    0.852565  0.853075

## 3.4 로지스틱 회귀 분석 (Logistic Regression)

In [21]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = 'liblinear')
lr.fit(x_train, y_train)

LogisticRegression(solver='liblinear')

In [ ]:
# 성능 평가
train_score = lr.score(x_train, y_train)
test_score = lr.score(x_test, y_test)
print(train_score, test_score)

In [22]:
# 평가 결과 score_df에 추가
score_df.loc['LogisticRegression'] = [train_score, test_score]
score_df

train      test
DecisionTree        0.981074  0.805369
RandomForest        0.981074  0.851503
NaiveBayes          0.852565  0.853075
LogisticRegression  0.852565  0.853075

## 3.5 SVM (Support Vector Machine)

In [23]:
from sklearn.svm import LinearSVC
svc = LinearSVC(verbose=True)
svc.fit(x_train, y_train)

[LibLinear]

LinearSVC(verbose=True)

In [24]:
# 성능 평가
train_score = svc.score(x_train, y_train)
test_score = svc.score(x_test, y_test)
print(train_score, test_score)

0.861493499949612 0.8599068867525244


In [25]:
# 평가 결과 score_df에 추가
score_df.loc['SVM'] = [train_score, test_score]
score_df

train      test
DecisionTree        0.981074  0.805369
RandomForest        0.981074  0.851503
NaiveBayes          0.852565  0.853075
LogisticRegression  0.852565  0.853075
SVM                 0.861493  0.859907

## 3.6 Perceptron

In [26]:
from sklearn.linear_model import Perceptron

perc = Perceptron()
perc.fit(x_train, y_train)

Perceptron()

In [27]:
# 성능 평가
train_score = perc.score(x_train, y_train)
test_score = perc.score(x_test, y_test)
print(train_score, test_score)

0.8070543182505291 0.8060342221416047


In [28]:
# 평가 결과 score_df에 추가
score_df.loc['Perceptron'] = [train_score, test_score]
score_df

train      test
DecisionTree        0.981074  0.805369
RandomForest        0.981074  0.851503
NaiveBayes          0.852565  0.853075
LogisticRegression  0.852565  0.853075
SVM                 0.861493  0.859907
Perceptron          0.807054  0.806034

## 3.7 성능 비교

In [29]:
# 평가 결과 저장 데이터 프레임 확인
score_df.sort_values(by='test', ascending=False)

train      test
SVM                 0.861493  0.859907
NaiveBayes          0.852565  0.853075
LogisticRegression  0.852565  0.853075
RandomForest        0.981074  0.851503
Perceptron          0.807054  0.806034
DecisionTree        0.981074  0.805369

## 4. 영화 리뷰 긍부정 판단
* 학습된 모델 중 선택하여 활용

In [ ]:
# 영화 리뷰감성 분석용 tokenizer 정의


In [35]:
review = '영화가 재미있다'

# 텍스트 전처리
tokens = my_tokenizer(review)
tokens = " ".join(tokens) # 이거 써야 됩니다
print(tokens)

# 특징 벡터 추출
x = tv.transform([tokens])

# 예측
pred = svc.predict(x)

# 예측 결과 출력
result = '긍정' if pred >= 0.5 else '부정'
print(result)

영화 재미있다
긍정


In [41]:
# 함수로 만들기
def semtiment_analysis(review):
    # 텍스트 전처리
    tokens = my_tokenizer(review)
    tokens = " ".join(tokens) # 이거 써야 됩니다
    
    # 특징 벡터 추출
    x = tv.transform([tokens])
    
    # 예측
    pred = lr.predict(x)
    
    # 예측 결과 출력
    result = '긍정' if pred >= 0.5 else '부정'
    return result

In [42]:
# 함수 테스트
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '하품만 나온다',
    '이 영화 핵노잼 ㅠㅠ',
    '이딴게 영화냐 ㅉㅉ',
    '와 개쩐다',
    '감독 뭐하는 놈이냐',
    '정말 세계관 최강자들의 영화다'
]

for review in reviews:
    result = semtiment_analysis(review)
    print(f'{review} -> {result}')

이 영화 개꿀잼 ㅋㅋㅋ -> 부정
하품만 나온다 -> 긍정
이 영화 핵노잼 ㅠㅠ -> 부정
이딴게 영화냐 ㅉㅉ -> 부정
와 개쩐다 -> 긍정
감독 뭐하는 놈이냐 -> 부정
정말 세계관 최강자들의 영화다 -> 부정


In [ ]:
# 문장을 입력 받아서 긍부정 판단 
